# Histopathologic Cancer Detection

# Imports and Paths

In [1]:
import numpy as np
import pandas as pd
import os
import cv2
import matplotlib.pyplot as plt
import matplotlib.patches as patches
import random
from sklearn.utils import shuffle
from tqdm import tqdm_notebook

# fastai 1.0
from fastai import *
from fastai.vision import *
from torchvision.models import *    # import *=all the models from torchvision  
import torchvision.transforms as transforms

CROP_SIZE = 90 
arch = densenet169                  # specify model architecture, densenet169 seems to perform well for this data but you could experiment
BATCH_SIZE = 128                    # specify batch size, hardware restrics this one. Large batch sizes may run out of GPU memory
sz = CROP_SIZE                      # input size is the crop size
MODEL_PATH = str(arch).split()[1]   # this will extrat the model name as the model file name e.g. 'resnet50'

MODEL_NAME = "densenet169_wsi_auc_aug_focal_tta_head"

data = pd.read_csv('/home/josen/datasets/histopathologic-cancer-detection/train_labels.csv')
train_path = '/home/josen/datasets/histopathologic-cancer-detection/train/'
test_path = '/home/josen/datasets/histopathologic-cancer-detection/test/'
# quick look at the label stats
data['label'].value_counts()

0    130908
1     89117
Name: label, dtype: int64

# Data Preparation

In [2]:
df_WSI=pd.read_csv('/home/josen/datasets/histopathologic-cancer-detection/patch_id_wsi.csv')
df_trn=data

df_notinWSI=df_trn.set_index('id').drop(df_WSI.id)
valWSI=df_WSI.groupby(by='wsi')['id'].count().sample(frac=0.23).index
trnWSI=[i[0] for i in df_WSI.groupby(by='wsi')['id'] if i[0] not in valWSI]

val_idx=np.hstack([df_WSI.groupby(by='wsi')['id'].indices[WSI] for WSI in valWSI])
val_idx=np.append(df_notinWSI.index.values,df_WSI.id[val_idx])
trn_idx=np.hstack([df_WSI.groupby(by='wsi')['id'].indices[WSI] for WSI in trnWSI])
trn_idx=df_WSI.id[trn_idx]

val_idx=df_trn.reset_index().set_index('id').loc[val_idx,'index'].values
trn_idx=df_trn.reset_index().set_index('id').loc[trn_idx,'index'].values

np.random.shuffle(val_idx)
np.random.shuffle(trn_idx)

# create test dataframe
test_names = []
for f in os.listdir(test_path):
    test_names.append(test_path + f)
df_test = pd.DataFrame(np.asarray(test_names), columns=['name'])
    
src = (ImageList.from_df(df_trn, path=train_path, suffix='.tif',folder='')                
                .split_by_idxs(trn_idx,val_idx)
                .label_from_df(label_delim=' '))
src.add_test(ImageList.from_df(path='/', df=df_test))

LabelLists;

Train: LabelList (145229 items)
x: ImageList
Image (3, 96, 96),Image (3, 96, 96),Image (3, 96, 96),Image (3, 96, 96),Image (3, 96, 96)
y: MultiCategoryList
0,0,1,1,1
Path: /home/josen/datasets/histopathologic-cancer-detection/train;

Valid: LabelList (74796 items)
x: ImageList
Image (3, 96, 96),Image (3, 96, 96),Image (3, 96, 96),Image (3, 96, 96),Image (3, 96, 96)
y: MultiCategoryList
0,0,1,0,0
Path: /home/josen/datasets/histopathologic-cancer-detection/train;

Test: LabelList (57458 items)
x: ImageList
Image (3, 96, 96),Image (3, 96, 96),Image (3, 96, 96),Image (3, 96, 96),Image (3, 96, 96)
y: EmptyLabelList
,,,,
Path: /home/josen/datasets/histopathologic-cancer-detection/train

# Data Augmentation

In [3]:
tfms = get_transforms(do_flip=True, flip_vert=True, max_rotate=0.0, max_zoom=.15,
                      max_lighting=0.1, max_warp=0.15)
tfms[1].append(dihedral())

print(tfms[1])

data_bunch=ImageDataBunch.create_from_ll(src, ds_tfms=tfms, size=sz,bs=BATCH_SIZE)
stats=data_bunch.batch_stats()       
data_bunch.normalize(stats);

[RandTransform(tfm=TfmCrop (crop_pad), kwargs={}, p=1.0, resolved={}, do_run=True, is_random=True), RandTransform(tfm=TfmPixel (dihedral), kwargs={}, p=1.0, resolved={}, do_run=True, is_random=True)]


In [4]:
from sklearn.metrics import roc_auc_score
def auc_score(y_pred,y_true,tens=True):
    score=roc_auc_score(y_true[:,1],torch.sigmoid(y_pred)[:,1])
    if tens:
        score=tensor(score)
    else:
        score=score
    return score

In [5]:
class FocalLoss(nn.Module):
    def __init__(self, alpha=1, gamma=2, logits=False, reduce=True):
        super(FocalLoss, self).__init__()
        self.alpha = alpha
        self.gamma = gamma
        self.logits = logits
        self.reduce = reduce
    def forward(self, inputs, targets):
        if self.logits:
            BCE_loss = F.binary_cross_entropy_with_logits(inputs, targets, reduce=False)
        else:
            BCE_loss = F.binary_cross_entropy(inputs, targets, reduce=False)
        pt = torch.exp(-BCE_loss)
        F_loss = self.alpha * (1-pt)**self.gamma * BCE_loss

        if self.reduce:
            return torch.mean(F_loss)
        else:
            return F_loss

In [6]:
class CustomLinearHead(nn.Module):
    def __init__(self, pretrained=True):
        super().__init__()
        self.linear = nn.Linear(1000+2, 16)
        self.bn = nn.BatchNorm1d(16)
        self.dropout = nn.Dropout(0.2)
        self.elu = nn.ELU()
        self.out = nn.Linear(16, 1)
    
    def forward(self, x):
        out = x
        #batch = out.shape[0]
        max_pool, _ = torch.max(out, 1, keepdim=True)
        avg_pool = torch.mean(out, 1, keepdim=True)

        print(out.shape)
        #out = out.view(batch, -1)
        conc = torch.cat((max_pool, avg_pool), 1)

        conc = self.linear(conc)
        conc = self.elu(conc)
        conc = self.bn(conc)
        conc = self.dropout(conc)

        res = self.out(conc)

        return res

In [7]:
# Next, we create a convnet learner object
# ps = dropout percentage (0-1) in the final layer

customHead = CustomLinearHead()
def getLearner():
    return create_cnn(data_bunch, 
                      arch, 
                      pretrained=True, 
                      path='.', 
                      metrics=[auc_score], 
                      ps=0.5, 
                      callback_fns=ShowGraph, 
                      loss_func=FocalLoss(logits=True,gamma=1),
                      custom_head = customHead
                     )

learner = getLearner()

/home/josen/py-36-env/lib/python3.6/site-packages/fastai/vision/learner.py:105: UserWarning: `create_cnn` is deprecated and is now named `cnn_learner`.
  warn("`create_cnn` is deprecated and is now named `cnn_learner`.")


In [9]:
max_lr = 2e-2
wd = 1e-4
# 1cycle policy
learner.fit_one_cycle(cyc_len=8, max_lr=max_lr, wd=wd)

epoch,train_loss,valid_loss,auc_score,time


torch.Size([128, 1664, 2, 2])


RuntimeError: size mismatch, m1: [512 x 2], m2: [1002 x 16] at /pytorch/aten/src/THC/generic/THCTensorMathBlas.cu:266

In [ ]:
# plot learning rate of the one cycle
learner.recorder.plot_lr()

In [ ]:
# and plot the losses of the first cycle
learner.recorder.plot_losses()

In [ ]:
# before we continue, lets save the model at this stage
learner.save(MODEL_PATH + MODEL_NAME + '_stg1')

In [ ]:
# load the baseline model
learner.load(MODEL_PATH + MODEL_NAME + '_stg1')

# unfreeze and run learning rate finder again
learner.unfreeze()
learner.lr_find(wd=wd)

# plot learning rate finder results
learner.recorder.plot()

In [ ]:
# Now, smaller learning rates. This time we define the min and max lr of the cycle
learner.fit_one_cycle(cyc_len=12, max_lr=slice(4e-5,4e-4))

In [ ]:
# Save the finetuned model
learner.save(MODEL_PATH + MODEL_NAME + '_stg2')

# Results
 * train_loss valid_loss valid_acc
 * Densenet169+OneCycle+NoAugmentation = 0.002263	0.376988 0.970504
 * Densenet169+OneCycle+Augmentation = 0.092341	0.097588	0.969959

# Validation and Submission

In [ ]:
# make sure we have the best performing model stage loaded
print(MODEL_NAME)
learner.load(MODEL_PATH + MODEL_NAME + '_stg2')

# Fastai has a function for this but we don't want the additional augmentations it does (our image loader has augmentations) so we just use the get_preds
preds_test,y_test=learner.TTA(ds_type=DatasetType.Test)

# We do a fair number of iterations to cover different combinations of flips and rotations.
# The predictions are then averaged.
#n_aug = 12
#preds_n_avg = np.zeros((len(learner.data.test_ds.items),2))
#for n in tqdm_notebook(range(n_aug), 'Running TTA...'):
#    preds,y = learner.get_preds(ds_type=DatasetType.Test, with_loss=False)
#    preds_n_avg = np.sum([preds_n_avg, preds.numpy()], axis=0)
#preds_n_avg = preds_n_avg / n_aug

In [ ]:
n_aug = 12
preds_n_avg = np.zeros((len(learner.data.test_ds.items),2))

preds_n_avg = np.sum([preds_n_avg, preds_test.numpy()], axis=0)
preds_n_avg = preds_n_avg / n_aug
print(preds_n_avg)

# Next, we will transform class probabilities to just tumor class probabilities
print('Negative and Tumor Probabilities: ' + str(preds_n_avg[0]))
tumor_preds = preds_n_avg[:, 1]
print('Tumor probability: ' + str(tumor_preds[0]))
# If we wanted to get the predicted class, argmax would get the index of the max
class_preds = np.argmax(preds_n_avg, axis=1)
classes = ['Negative','Tumor']
print('Class prediction: ' + classes[class_preds[0]])

In [ ]:
# get test id's from the sample_submission.csv and keep their original order
SAMPLE_SUB = '/home/josen/datasets/histopathologic-cancer-detection/sample_submission.csv'
sample_df = pd.read_csv(SAMPLE_SUB)
sample_list = list(sample_df.id)

# List of tumor preds. 
# These are in the order of our test dataset and not necessarily in the same order as in sample_submission
pred_list = [p for p in tumor_preds]

# To know the id's, we create a dict of id:pred
pred_dic = dict((key, value) for (key, value) in zip(learner.data.test_ds.items, pred_list))

# Now, we can create a new list with the same order as in sample_submission
pred_list_cor = [pred_dic['///home/josen/datasets/histopathologic-cancer-detection/test/' + id + '.tif'] for id in sample_list]

# Next, a Pandas dataframe with id and label columns.
df_sub = pd.DataFrame({'id':sample_list,'label':pred_list_cor})

# Export to csv
df_sub.to_csv('{0}_submission.csv'.format(MODEL_PATH + MODEL_NAME), header=True, index=False)

In [ ]:
print(MODEL_PATH + MODEL_NAME)